# Ipynb to 

graph and try out different 

ML concepts with a sample datasize.


A partir de las 3 tablas ya procesdas : 
* homeantennas
* sumlinks
* groundtruth

armames un datset para explorar conceptos de ML

In [1]:
## seteamos los parametros del notebook
%autosave 180
import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np; import os;import random;
import graphlab as gl
from IPython.display import display # para pretty-print con estetica ipython si es que estamos dentro de  un loop, if, etc
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#seteamos el lugar de trabajo
HOMEDIR=os.path.expanduser('~')

PROJECTDIR = os.getcwd().split(os.sep)
PROJECTDIR =  os.sep.join(PROJECTDIR[:PROJECTDIR.index('mexico-scripts-ver2') + 1])

DATADIR = os.path.join(PROJECTDIR,'datasets')

DATADIR2 = os.path.join(PROJECTDIR,'data')
# os.chdir(DATADIR)


Autosaving every 180 seconds


In [2]:
!ls $DATADIR

data_balanced_sample.csv  gtruth_0215_0715  sl
gtruth_0114_0715	  homeant	    sl.csv.gz


In [3]:
graphlab_frame = False
# graphlab_frame = 'juan' in HOMEDIR

def get_input_sframe(ds,graphlab_frame=graphlab_frame):
    if graphlab_frame:
        if ds == 'home_antenna': url = DATADIR +"/homeant"
        elif ds == 'sum_links': url  = DATADIR +"/sl"
        elif ds == 'gtruth_02': url  = DATADIR +"/gtruth_0215_0715" # can be near ground truth (previous to july 2015)
        elif ds == 'gtruth_01': url  =  DATADIR +"/gtruth_0114_0715" # or can be old GT
        else: print('type chosen is %s, type should be home_antenna, sum_links, gtruth_02 or gtruth_01' % ds)
    else:
        url = DATADIR + '/data_balanced_sample.csv'
    return url 

In [4]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

## decide if load a reduced size dataset

In [5]:
# set what is the resulting sample we want of the global dataset
global_split = 0.25
global_split = None

# since we have 2 datets that will be later joined and which are going to be previously sampled, in the end this 
# independent sampling will result in that, after the join, the dataset will have a size of fraction given by 
# the `global_split` variable
if global_split:
    
    seed = 2015
    sample = np.sqrt(global_split)
    print(sample)

In [6]:
%%time

if graphlab_frame:
    sf_table, _ = gl.load_sframe(get_input_sframe('home_antenna',graphlab_frame))
    sl_table, _ = gl.load_sframe(get_input_sframe('sum_links',graphlab_frame))
    # no need to sample the gt table since it is very small in memory
    gt_table = gl.load_sframe(get_input_sframe('gtruth_01',graphlab_frame))

    if global_split:

        sf_table, _ = sf_table.random_split(sample,seed=seed)
        sl_table, _ = sl_table.random_split(sample,seed=seed)


    rename_gt = (dict([(col,col+"_gt") for col in gt_table.column_names() if col != 'USER']))

    #agrego la etiqueta "_gt" a las columnas del ground_truth
    gt_table.rename(rename_gt) 
else:
    data = pd.read_csv(get_input_sframe('sum_links',graphlab_frame))

CPU times: user 3.84 s, sys: 224 ms, total: 4.07 s
Wall time: 4.7 s


In [7]:
if graphlab_frame:
    print(sf_table.shape, sl_table.shape, gt_table.shape)
else:
    print(data.shape)

(153010, 176)


## simple format description

Aca miramos las top 10 antenas que utilizo un usuario, ordeandas de 0 a 10 segun el uso, en numero de llamados, que el user le daba a c. antenna. _0_ Seria la antenna mas utilizada y _10_ la menos utilizada. El count es la cantidad de veces que utilizo esa antenna. 

Cuando aparece _WEEKNIGHT_ en el nombre de la columna, es porque cumple la condicion de que los llamados fueron hechos de noche fuera del horario [8,20] y dentro de la semana laboral.

Siguiendo las definiciones del trabajo de Caro, un user es _EPIDEMIC_ siii su ANTENNA_WEEKNIGHT_0 (esta es la home_antenna) pertence a la zona epidemica.

El mobility_diameter es el radio de las antennas (0 si uso una sola, etc.) utilizadas por este user. Nuevamente el modificador _WEEKNIGHT_ solo aplica para antennas utilizadas en esos horarios.

In [8]:
# column names
if graphlab_frame:
    print(sf_table.column_names())
else:
    print(data.columns)

Index([u'USER', u'COUNT_0', u'COUNT_1', u'COUNT_2', u'COUNT_3', u'COUNT_4',
       u'COUNT_5', u'COUNT_6', u'COUNT_7', u'COUNT_8',
       ...
       u'TimeWeekDay_IN_11', u'CallsWeekEnd_IN_11', u'TimeWeekNight_IN_11',
       u'TimeWeekEnd_IN_VUL_11', u'CallsWeekDay_IN_VUL_11',
       u'CallsWeekNight_IN_VUL_11', u'TimeWeekDay_IN_VUL_11',
       u'CallsWeekEnd_IN_VUL_11', u'TimeWeekNight_IN_VUL_11',
       u'VULNERABLE_IN_11'],
      dtype='object', length=176)


## sum links description

La tabla sum_links tiene mas atributos y con data mas rica en informacion:
    
Esencialmente para c/ user miramos la cantidad de llamados y el tiempo que duraron esos llamados pero segmentando con distintos modificadores. `Time` representa la duracion del llamado y Calls el conteo de llamados.

Los modficadores/ segmentaciones son:

* mes en el cual estamos parados (12 == diciembre, ..., 08 == agosto)
* OUT/IN, separa por la direccion de los llamados si salientes u entrantes.
* _VULN_ : separa los llamados que fueron realizados hacia/desde un target_user (en una llamada hay 2 usuarios, el origin o el target) viviendo en una zona epidemica. Donde la home antena de un target_user determina su vulnerabilidad segun si es zona epidemica o no.
* Weekend, WeekDay y WeekNight son lo que suenan. Weekend el finde, Weeknight la semana pero fuera de horario laboral y Weekday en horario laboral y de lunes a viernes.

Hay solo una columna que no entra enteramente en este esquema que es VULNERABLE. Esta columna hace un conteo p/c/ usuario d cuantos target_users viven en una zona epidemica. Tambien se segmenta esta columna con los modficiadores anteriores (el mes y el out/in).

In [10]:
# column names
if graphlab_frame:
    print(sl_table.column_names())

# Para la
tabla gt (ground_truth) es mas simple la explicacion. Solo se busco el antenna_ID_0 (nuevamente la antenna mas utilizada) por un user

In [11]:
if graphlab_frame:
    print(gt_table.head())

# We need antennas metadata
to get the epidemicity of each antenna and add that info to the dataset

In [12]:
DATADIR2

'/home/juan/mobility-study/mexico-scripts-ver2/data'

In [13]:
DATADIR

'/home/juan/mobility-study/mexico-scripts-ver2/datasets'

In [14]:
#agregamos al gt su info de epidemicidad
antennas_file = DATADIR2+'/celdas_limpio.csv'

if graphlab_frame:
    antennas = gl.SFrame.read_csv(antennas_file, 
                              delimiter= "|", 
                usecols=['LATITUDE','LONGITUDE','CEL_ID','STATE','EPIDEMIC'],
                column_type_hints=[float, float, str,str, bool]
                            )
else:
    antennas = pd.read_csv(antennas_file,sep='|')


In [15]:
%%time
# enriquecemos la data de gt con info de epidemicidad
if graphlab_frame:
    # agrego tambien data de epidemicidad al simpleformat table
    sf_table = sf_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                             on = {'ANTENNA_ID_WEEKNIGHT_0':'CEL_ID'},
                             how = 'left')
    gt_table = gt_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                             on = {'ANTENNA_ID_WEEKNIGHT_0_gt':'CEL_ID'},
                             how = 'left')

    gt_table.rename({'EPIDEMIC':'EPIDEMIC_gt'});

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


# create data matrix

In [16]:
%%time
if graphlab_frame:
    data = sl_table.join(gt_table['USER','EPIDEMIC_gt'], on = 'USER', how = 'inner')
    data = data.join(sf_table, on = 'USER', how = 'inner')
    #no podemos tener nulls en el target asi que dropeamos
    data = data.dropna(columns = ['EPIDEMIC_gt'], how='any')
    del sl_table, sf_table, gt_table

print(data.shape)


(153010, 176)
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 859 µs


In [17]:
data['EPIDEMIC_gt'].sum(),data['EPIDEMIC'].sum()

(51709, 47344)

In [18]:
# data.query('EPIDEMIC==0  ').STATE.value_counts()

## preparamos los datasets que no pueden tomar valores negativos o categorical vars

# train test split as 
p% of set as validation and the resulting  as train

In [19]:
split_perc = 0.9
mask = np.random.rand(data.shape[0])< split_perc

val_set = data[mask==0]
data = data[mask==1]


## define our real X variable and Y vars

exclude/include features. Decide our problem (multi-target, single_target)

## in our  X features..

determine which columns have no meaning.

we are going to try different models

In [125]:
if graphlab_frame: iterable=data.column_names()
else: iterable=data.columns

manual_exclude_cols = [     
     'EPIDEMIC',
      'EPIDEMIC_gt',
#       'STATE',
                ]

comprehensive_exclude_cols = [col for col in iterable if col == 'USER' 
          or ('ANTENNA' in col) ]   
                            
    
exclude_cols = manual_exclude_cols + comprehensive_exclude_cols
exclude_cols

['EPIDEMIC',
 'EPIDEMIC_gt',
 'USER',
 'ANTENNA_ID_0',
 'ANTENNA_ID_1',
 'ANTENNA_ID_2',
 'ANTENNA_ID_3',
 'ANTENNA_ID_4',
 'ANTENNA_ID_5',
 'ANTENNA_ID_6',
 'ANTENNA_ID_7',
 'ANTENNA_ID_8',
 'ANTENNA_ID_9',
 'ANTENNA_ID_WEEKNIGHT_0',
 'ANTENNA_ID_WEEKNIGHT_1',
 'ANTENNA_ID_WEEKNIGHT_2',
 'ANTENNA_ID_WEEKNIGHT_3',
 'ANTENNA_ID_WEEKNIGHT_4',
 'ANTENNA_ID_WEEKNIGHT_5',
 'ANTENNA_ID_WEEKNIGHT_6',
 'ANTENNA_ID_WEEKNIGHT_7',
 'ANTENNA_ID_WEEKNIGHT_8',
 'ANTENNA_ID_WEEKNIGHT_9']

## first our Y vars

define them with a set of different possible cases/problems to solve

In [142]:
case = 0

## people that used to live in the endemic area
if case ==0:
    case_text = "people that used to live in the endemic area"
    Y = data['EPIDEMIC_gt'] == 1
    Y_val = val_set['EPIDEMIC_gt'] == 1

## people that used to live in the endemic area *and* migrated
if case ==1:
    case_text = "people that used to live in the endemic area *and* migrated"
    Y = (data['EPIDEMIC_gt'] ==1) & (data['EPIDEMIC'] ==0)
    Y_val = (val_set['EPIDEMIC_gt'] ==1) & (val_set['EPIDEMIC'] ==0)
    
    add_exclusion_cols = ['EPIDEMIC','STATE']

    for col in add_exclusion_cols:
        if not col in exclude_cols:
            exclude_cols+=[col]

## people that migrated in any direction
if case ==2:
    case_text = "people that migrated in any direction"
    Y = data['EPIDEMIC_gt'] != data['EPIDEMIC']
    Y_val = val_set['EPIDEMIC_gt'] != data['EPIDEMIC']
    
    add_exclusion_cols = []

    for col in add_exclusion_cols:
        if not col in exclude_cols:
            exclude_cols+=[col]

    
## people that migrated in any direction
if case ==3:
    case_text = "people that migrated in any direction"
    Y = data['EPIDEMIC_gt'] != data['EPIDEMIC']
    Y_val = val_set['EPIDEMIC_gt'] != data['EPIDEMIC']
    
    add_exclusion_cols = []

    for col in add_exclusion_cols:
        if not col in exclude_cols:
            exclude_cols+=[col]

    
    
    
## people from the Mexico or DF states
if case == 4:
    case_text = "people from the Mexico or DF states"
    Y = (data['STATE'] == 'Distrito_Federal') | (data['STATE'] == 'Mexico')
    Y_val = (val_set['STATE'] == 'Distrito_Federal') | (val_set['STATE'] == 'Mexico')
    
    add_exclusion_cols = ['EPIDEMIC','STATE']

    for col in add_exclusion_cols:
        if not col in exclude_cols:
            exclude_cols+=[col]

In [143]:
if graphlab_frame: iterable=data.column_names()
else: iterable=data.columns

In [144]:
width = 30
N= np.random.randint(1,int(iterable.shape[0]*1.0/width))
data.columns[(N)*width: (N+1)*width]

Index([u'CallsWeekDay_IN_12', u'CallsWeekNight_IN_12', u'TimeWeekDay_IN_12',
       u'CallsWeekEnd_IN_12', u'TimeWeekNight_IN_12', u'TimeWeekEnd_IN_VUL_12',
       u'CallsWeekDay_IN_VUL_12', u'CallsWeekNight_IN_VUL_12',
       u'TimeWeekDay_IN_VUL_12', u'CallsWeekEnd_IN_VUL_12',
       u'TimeWeekNight_IN_VUL_12', u'VULNERABLE_IN_12', u'TimeWeekEnd_OUT_08',
       u'CallsWeekDay_OUT_08', u'CallsWeekNight_OUT_08', u'TimeWeekDay_OUT_08',
       u'CallsWeekEnd_OUT_08', u'TimeWeekNight_OUT_08',
       u'TimeWeekEnd_OUT_VUL_08', u'CallsWeekDay_OUT_VUL_08',
       u'CallsWeekNight_OUT_VUL_08', u'TimeWeekDay_OUT_VUL_08',
       u'CallsWeekEnd_OUT_VUL_08', u'TimeWeekNight_OUT_VUL_08',
       u'VULNERABLE_OUT_08', u'TimeWeekEnd_IN_08', u'CallsWeekDay_IN_08',
       u'CallsWeekNight_IN_08', u'TimeWeekDay_IN_08', u'CallsWeekEnd_IN_08'],
      dtype='object')

In [145]:
%%time

X = data[[col for col in iterable if col not in exclude_cols]]

X_val = val_set[[col for col in iterable if col not in exclude_cols]]

if graphlab_frame: iterable=X.column_names()
else: iterable=X.columns
    
# clean negative/Null vals in count cols 
for col in [col for col in iterable if 'COUNT' in col]:
    X[col]= X[col].apply(lambda x :  x if x>=0 else 0)
    X_val[col]= X_val[col].apply(lambda x :  x if x>=0 else 0)



/home/juan/mfixman/venv/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/juan/mfixman/venv/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


CPU times: user 5.09 s, sys: 184 ms, total: 5.28 s
Wall time: 5.26 s


In [146]:
X.shape, Y.shape

((137593, 153), (137593,))

## dummy-ize categorical cols
if necessary

In [147]:
if graphlab_frame: iterable=X.column_names()
else: iterable=X.columns


In [148]:
categorical_cols = [
    'STATE'
]

In [149]:
for col in categorical_cols:
    if col not in exclude_cols:
        X[col] = X[col].astype('category')
    #     if X[col].dtype != 'category':
    #         continue 
        X = pd.concat([X,pd.get_dummies(X[col], 
                                          prefix= col, 
                                          prefix_sep='_', 
                                          #sparse = True,
                                          dummy_na=False).astype(np.int8)],\
                  axis=1 ,join = 'inner')
        X.drop(col, axis =1 , inplace=True)

        # now onto test_table
        X_val[col] = X_val[col].astype('category')
        X_val = pd.concat([X_val,pd.get_dummies(X_val[col], 
                                          prefix= col, 
                                          prefix_sep='_', 
                                          #sparse = True,
                                          dummy_na=False).astype(np.int8)],\
                  axis=1 ,join = 'inner')

        X_val.drop(col, axis =1 , inplace=True)


/home/juan/mfixman/venv/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/juan/mfixman/venv/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## explore correlations to target var

with current column configuration

In [150]:
corr = pd.DataFrame(X).copy()
target_col = 'target'
corr[target_col] = Y
corr = corr.corr()
corr.shape

(185, 185)

In [151]:
view = corr.query('target>0.3')
# show only those columns which 
corr_columns = view.index.values

view[corr_columns]

,VULNERABLE_OUT_12,VULNERABLE_IN_12,CallsWeekEnd_OUT_VUL_08,VULNERABLE_OUT_08,CallsWeekEnd_IN_VUL_08,VULNERABLE_IN_08,VULNERABLE_OUT_09,VULNERABLE_IN_09,VULNERABLE_OUT_10,VULNERABLE_IN_10,VULNERABLE_OUT_11,VULNERABLE_IN_11,target
VULNERABLE_OUT_12,1.000000,0.850295,0.378955,0.676784,0.382475,0.668981,0.708142,0.696803,0.748884,0.736026,0.784809,0.771825,0.504878
VULNERABLE_IN_12,0.850295,1.000000,0.378183,0.667287,0.386080,0.673075,0.698531,0.704119,0.737702,0.747121,0.769917,0.780831,0.502233
CallsWeekEnd_OUT_VUL_08,0.378955,0.378183,1.000000,0.434128,0.807458,0.425652,0.415937,0.411292,0.398278,0.397624,0.391708,0.389172,0.305565
VULNERABLE_OUT_08,0.676784,0.667287,0.434128,1.000000,0.434624,0.853652,0.782903,0.766690,0.732152,0.720763,0.707354,0.697080,0.553433
CallsWeekEnd_IN_VUL_08,0.382475,0.386080,0.807458,0.434624,1.000000,0.441331,0.418110,0.419436,0.400317,0.405754,0.394837,0.397313,0.312782
VULNERABLE_IN_08,0.668981,0.673075,0.425652,0.853652,0.441331,1.000000,0.766349,0.776035,0.716255,0.725815,0.695712,0.704881,0.557346
VULNERABLE_OUT_09,0.708142,0.698531,0.415937,0.782903,0.418110,0.766349,1.000000,0.853446,0.781529,0.766989,0.746180,0.733659,0.536786
VULNERABLE_IN_09,0.696803,0.704119,0.411292,0.766690,0.419436,0.776035,0.853446,1.000000,0.763409,0.775521,0.732264,0.741109,0.539676
VULNERABLE_OUT_10,0.748884,0.737702,0.398278,0.732152,0.400317,0.716255,0.781529,0.763409,1.000000,0.854079,0.802529,0.786197,0.521621
VULNERABLE_IN_10,0.736026,0.747121,0.397624,0.720763,0.405754,0.725815,0.766989,0.775521,0.854079,1.000000,0.784826,0.795040,0.520825


In [152]:
## show correlation between state and target

if not 'STATE' in exclude_cols:    
    state_cols = [col for col in corr if 'STATE' in col]
    view = corr[state_cols + [target_col]]
    display(view.query('target > 0.01'))


,STATE_Aguascalientes,STATE_Baja_California,STATE_Baja_California_Sur,STATE_Campeche,STATE_Chiapas,STATE_Chihuahua,STATE_Coahuila_de_Zaragoza,STATE_Colima,STATE_Distrito_Federal,STATE_Durango,...,STATE_San_Luis_Potosi,STATE_Sinaloa,STATE_Sonora,STATE_Tabasco,STATE_Tamaulipas,STATE_Tlaxcala,STATE_Veracruz,STATE_Yucatan,STATE_Zacatecas,target
MOBILITY_DIAMETER,0.045310,0.146754,0.031551,0.005904,-0.013297,0.022532,-0.002428,-0.006847,-0.137172,-0.009170,...,0.061558,0.053276,0.078379,0.004430,0.005110,-0.020230,0.056143,-0.028402,-0.000032,0.015361
MOBILITY_DIAMETER_WEEKNIGHT,0.046512,0.144198,0.031054,0.005293,-0.009695,0.023714,-0.000556,-0.005593,-0.119614,-0.009337,...,0.062585,0.056015,0.080636,0.007443,0.005611,-0.024561,0.045788,-0.020893,-0.006505,0.016656
TimeWeekEnd_OUT_VUL_12,-0.013656,-0.005871,0.010332,-0.002648,-0.012006,-0.005701,-0.012091,0.007858,-0.058754,-0.009689,...,-0.017232,0.008689,0.011338,0.061751,-0.000276,-0.012801,0.025614,-0.025107,-0.005510,0.187150
CallsWeekDay_OUT_VUL_12,-0.022717,-0.016940,0.001280,-0.012753,-0.024743,-0.009175,-0.017692,0.008193,-0.095536,-0.013729,...,-0.029892,0.004333,0.008663,0.088112,-0.009520,-0.021678,0.043872,-0.040398,-0.011481,0.279418
CallsWeekNight_OUT_VUL_12,-0.019146,-0.010924,0.007763,-0.003611,-0.019226,-0.005163,-0.013926,0.008456,-0.067328,-0.010774,...,-0.022328,0.008362,0.010211,0.077341,-0.000436,-0.015107,0.038242,-0.031883,-0.008867,0.235352
TimeWeekDay_OUT_VUL_12,-0.017420,-0.010759,0.006340,-0.005034,-0.017892,-0.005831,-0.015003,0.014442,-0.071673,-0.011923,...,-0.025590,0.010060,0.012857,0.071836,-0.005827,-0.017348,0.032937,-0.033268,-0.009946,0.230834
CallsWeekEnd_OUT_VUL_12,-0.020489,-0.015170,0.004070,-0.010087,-0.021958,-0.009211,-0.016432,0.006108,-0.088074,-0.013220,...,-0.026724,0.003119,0.008908,0.085221,-0.007715,-0.018324,0.043660,-0.036891,-0.010454,0.267177
TimeWeekNight_OUT_VUL_12,-0.011112,-0.002904,0.012386,0.006997,-0.008463,0.000747,-0.007113,0.005586,-0.033304,-0.006289,...,-0.011730,0.014307,0.012259,0.045231,0.004583,-0.007925,0.017499,-0.018186,-0.004133,0.145748
VULNERABLE_OUT_12,-0.040005,-0.029371,0.009205,-0.017340,-0.040038,-0.015465,-0.034485,0.010054,-0.148220,-0.025432,...,-0.052034,0.017125,0.017521,0.117944,-0.014878,-0.026145,0.134381,-0.075027,-0.020206,0.504878
TimeWeekEnd_IN_VUL_12,-0.012306,-0.007129,0.009631,-0.000750,-0.008584,-0.002293,-0.010550,0.009163,-0.055680,-0.009105,...,-0.016060,0.007889,0.006321,0.061526,-0.001670,-0.011228,0.027358,-0.021962,-0.005557,0.177470


In [155]:
a=data['EPIDEMIC_gt'].sum();b= data.shape[0]
a*1.0/b

0.338018649204538

In [156]:
a=val_set['EPIDEMIC'].sum();b= val_set.shape[0]
a*1.0/b

0.30810144645521176

In [157]:
# cast to numpy types if necessary
# %%time
# X = X.to_dataframe()
# Y = pd.Series(data=Y.to_numpy(),dtype=np.bool,name = 'EPIDEMIC_gt')
# X_val = X_val.to_dataframe()
# Y_val = pd.Series(data=Y_val.to_numpy(),dtype=np.bool,name = 'EPIDEMIC_gt')

# Instance fast model with MNB
#### convertimos la data en numpy, previamente limpiando las columnas de antennas ya que el clasificador no acepta strings

In [158]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import *
from sklearn.metrics import *
# from sklearn.cross_validation import *
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV, 
                                     train_test_split, KFold, cross_val_predict, 
                                     cross_val_score, learning_curve, validation_curve
                                    )

## remember the Multinomail NB assumes no negative values
are present in the dataset as this must be `count` data

In [159]:
%%time
start_time = time.time()
param_grid = {'alpha':[1e-3, 1e-1, 1e1,1e0], 'fit_prior': [True,False],
             }

mnb  = MultinomialNB( )

clf = GridSearchCV(mnb, param_grid, scoring='f1_weighted', fit_params=None, n_jobs=-1, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf2 = MultinomialNB( )
#how many parameters to randomly search for
#n_iter_search = 45

#random_search = RandomizedSearchCV(rforest, param_distributions=param_grid,
                                 #  n_iter=n_iter_search, n_jobs =8, verbose=3)

elapsed_time =  time.time() - start_time

#Y = categorical(train_table_target.values, drop=True).astype(int)

clf.fit(X,Y)

elapsed_time =   time.time() - start_time 

print('Grid Search took %s seconds to run' % (elapsed_time))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1


#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('\n Best estimator was %s \n' % str(clf.best_estimator_))
print('\n Best estimator was %s \n' % str(clf.best_score_))

clf2.set_params(**clf.best_params_)

clf2.fit(X,Y)


#converted_dict = evaluation_print(clf, X_val, test_table_target.values, test_table.index.values, 
#                              test_table_target[test_table_target>0].index.values,start_date,future)

print(classification_report(Y_val,clf2.predict(X_val)))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] alpha=0.001, fit_prior=True .....................................
[CV] alpha=0.001, fit_prior=True .....................................
[CV] alpha=0.001, fit_prior=True .....................................
[CV] ...... alpha=0.001, fit_prior=True, score=0.785322, total=   0.7s
[CV] alpha=0.001, fit_prior=False ....................................
[CV] ...... alpha=0.001, fit_prior=True, score=0.957841, total=   0.7s


[Parallel(n_jobs=-1)]: Done   2 out of  24 | elapsed:    2.2s remaining:   24.4s


[CV] alpha=0.001, fit_prior=False ....................................
[CV] ...... alpha=0.001, fit_prior=True, score=0.746840, total=   0.8s
[CV] alpha=0.001, fit_prior=False ....................................
[CV] ..... alpha=0.001, fit_prior=False, score=0.785096, total=   0.7s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] ..... alpha=0.001, fit_prior=False, score=0.957841, total=   0.7s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] ..... alpha=0.001, fit_prior=False, score=0.746840, total=   0.8s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] ........ alpha=0.1, fit_prior=True, score=0.785322, total=   0.8s
[CV] alpha=0.1, fit_prior=False ......................................
[CV] ........ alpha=0.1, fit_prior=True, score=0.957841, total=   0.8s
[CV] alpha=0.1, fit_prior=False ......................................
[CV] ........ alpha=0.1, fit_prior=True, score=0.746840, total=   0.7s
[CV] a

[Parallel(n_jobs=-1)]: Done  11 out of  24 | elapsed:    6.5s remaining:    7.6s


[CV] ....... alpha=0.1, fit_prior=False, score=0.746840, total=   0.8s
[CV] alpha=10.0, fit_prior=True ......................................
[CV] ....... alpha=10.0, fit_prior=True, score=0.785302, total=   0.8s
[CV] alpha=10.0, fit_prior=False .....................................
[CV] ....... alpha=10.0, fit_prior=True, score=0.957841, total=   0.8s
[CV] alpha=10.0, fit_prior=False .....................................
[CV] ....... alpha=10.0, fit_prior=True, score=0.746840, total=   0.8s
[CV] alpha=10.0, fit_prior=False .....................................
[CV] ...... alpha=10.0, fit_prior=False, score=0.785096, total=   0.8s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] ...... alpha=10.0, fit_prior=False, score=0.957841, total=   0.8s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] ...... alpha=10.0, fit_prior=False, score=0.746840, total=   0.8s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] .

...

[CV] ........ alpha=1.0, fit_prior=True, score=0.746840, total=   0.8s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ....... alpha=1.0, fit_prior=False, score=0.785096, total=   0.7s
[CV] ....... alpha=1.0, fit_prior=False, score=0.957841, total=   0.7s
[CV] ....... alpha=1.0, fit_prior=False, score=0.746840, total=   0.7s


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   12.0s finished


Grid Search took 13.9949519634 seconds to run

 Best estimator was MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True) 


 Best estimator was 0.830000911959 



/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/m

             precision    recall  f1-score   support

      False       0.87      0.88      0.88     10217
       True       0.76      0.75      0.75      5200

avg / total       0.83      0.83      0.83     15417

CPU times: user 11.8 s, sys: 1.56 s, total: 13.3 s
Wall time: 14.5 s


In [160]:
roc_auc_score(Y_val,clf2.predict(X_val))

0.81434681827421873

In [178]:
Y.shape, Y.sum(), clf.best_params_

((137593,), 46509, {'alpha': 0.001, 'fit_prior': True})

In [162]:
Y_val.sum()*1.0/Y_val.size

0.33729000454044239

In [163]:
roc_auc_score(Y_val,clf2.predict(X_val))

0.81434681827421873

In [177]:
for col in cv_result.columns:
    if col != 'mean_score':
        df = cv_result.groupby(col)['mean_score'].mean().to_frame().copy()
        df.columns = ['mean']
        df['std'] = cv_result.groupby(col)['mean_score'].std()

        display(df)

,mean,std
alpha,,
0.001,0.829963,0.000053
0.100,0.829963,0.000053
1.000,0.829963,0.000053
10.000,0.829960,0.000049


,mean,std
fit_prior,,
False,0.829925,0.000000
True,0.829999,0.000003


## now graphlab models 

In [179]:
from graphlab import random_forest_classifier, boosted_trees_classifier, logistic_classifier, decision_tree_classifier
from graphlab.toolkits import cross_validation, model_parameter_search

# get kfolds

In [181]:
%%time
num_folds = 10

target_col = 'target'
if graphlab_frame:   
    X[target_col] = Y
    X_val[target_col] = Y_val
    kfold = cross_validation.KFold(
    X,num_folds=num_folds)
else:
    X_gl= gl.SFrame(X)
    Y_gl = gl.SArray(Y)
    X_val_gl= gl.SFrame(X_val)
    Y_val_gl = gl.SArray(Y_val)
    X_gl[target_col] = Y_gl
    X_val_gl[target_col] = Y_val_gl
    
    kfold = cross_validation.KFold(
    X_gl,num_folds=num_folds)

CPU times: user 8.59 s, sys: 792 ms, total: 9.38 s
Wall time: 6.38 s


## RandomForest 
HyperParams Grid Search

In [ ]:

param_grid = {
    "target":target_col,
    
    'metric': [
                'auc',
                'log_loss',
                 ], 
    
    'max_iterations': [
                    50,
                    100,
                    150,
                    200,
                    ],
    
  'column_subsample': [
                  np.sqrt(X_gl.shape[1])/X_gl.shape[1],
                  np.log2(X_gl.shape[1])/X_gl.shape[1],
                  0.5,
                  0.1,
                  ], 
#     "bootstrap": [ 
#                 False,
#                 True,
#                     ],
#     "min_child_weight": np.append(np.random.randint(3,15,3),[3]),
    'max_depth':[
                 3,
                 6,
                 9,
                 12,
                ], 
   "class_weights": [
#                    'balanced',
                   None,
                   ],
#     "validation_set": X_val_gl,
              }

model_factory = random_forest_classifier.create

In [ ]:
%%time
start_time = time.time()
model_search = model_parameter_search.create( datasets = kfold, 
                                             #[data, val_set],
                                             model_factory = model_factory,
                                            model_parameters = param_grid, 
                                             perform_trial_run = True,
                                            )

search_results = model_search.get_results()

all_time = start_time - time.time()

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-May-25-2017-18-16-0900000' ready for execution


In [ ]:
res = model_search.get_metrics()
res

In [ ]:
search_results.shape

In [ ]:
for col in [col for col in search_results.column_names() if \
            not('accuracy' in col) and col!= 'model_id' and col!= 'random_seed' and col!= 'fold_id' \
                               and col!= 'num_folds']:
    print(search_results.groupby(col,
                             {'mean':gl.aggregate.MEAN('mean_training_accuracy'),
                                'std':gl.aggregate.STD('mean_training_accuracy')}))

### Refit best model for feature importance

In [ ]:
params = model_search.get_best_params()

In [ ]:
params

In [40]:
## manually insert params if needed
params = {'class_weights': None,
 'column_subsample': 0.5,
 'max_depth': 12,
 'max_iterations': 150,
 'metric': 'auc',
 'min_child_weight': 8,
 'min_loss_reduction': 10,
 'row_subsample': 0.9,
 'target': target_col}

In [41]:
%%time
best_model =  random_forest_classifier.create(dataset=X_gl,
                                             validation_set = X_val_gl,
                                             **params)

Random forest classifier:

--------------------------------------------------------

Number of examples          : 137593

Number of classes           : 2

Number of feature columns   : 185

...

| 140       | 117.967483   | 0.915319     | 0.903117       |

| 145       | 122.249418   | 0.915347     | 0.903164       |

| 150       | 126.400442   | 0.915416     | 0.903235       |

+-----------+--------------+--------------+----------------+

CPU times: user 10min 15s, sys: 12.7 s, total: 10min 28s
Wall time: 2min 13s


In [42]:
best_feat = best_model.get_feature_importance()
best_feat.head(20)

name,index,count
EPIDEMIC,None,2036
COUNT_0,None,1987
MOBILITY_DIAMETER,None,1914
MOBILITY_DIAMETER_WEEKNIGHT ...,None,1853
COUNT_WEEKNIGHT_0,None,952
TimeWeekDay_OUT_09,None,599
CallsWeekDay_IN_VUL_08,None,560
TimeWeekDay_OUT_08,None,555
STATE_Hidalgo,None,543
CallsWeekDay_IN_08,None,526


In [103]:
# filter_cols

In [100]:
best_feat['name'].head(30)
#print_rows(num_rows= 30)

dtype: str
Rows: 30
['EPIDEMIC', 'COUNT_0', 'MOBILITY_DIAMETER', 'MOBILITY_DIAMETER_WEEKNIGHT', 'COUNT_WEEKNIGHT_0', 'TimeWeekDay_OUT_09', 'CallsWeekDay_IN_VUL_08', 'TimeWeekDay_OUT_08', 'STATE_Hidalgo', 'CallsWeekDay_IN_08', 'TimeWeekDay_OUT_12', 'TimeWeekDay_OUT_VUL_08', 'CallsWeekDay_OUT_12', 'CallsWeekDay_OUT_09', 'CallsWeekDay_OUT_VUL_08', 'TimeWeekDay_OUT_11', 'COUNT_1', 'CallsWeekDay_OUT_08', 'TimeWeekDay_IN_12', 'CallsWeekDay_IN_12', 'TimeWeekDay_IN_09', 'TimeWeekDay_IN_11', 'CallsWeekDay_IN_09', 'TimeWeekDay_OUT_10', 'TimeWeekDay_IN_VUL_09', 'TimeWeekDay_OUT_VUL_12', 'STATE_Distrito_Federal', 'TimeWeekDay_IN_VUL_08', 'TimeWeekDay_IN_10', 'CallsWeekDay_OUT_10']

In [43]:
best_model.training_time

126.401433

In [44]:
test_error = best_model.evaluate(X_val_gl)

test_error

{'accuracy': 0.8488681325809172,
 'auc': 0.9032349929604522,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1048 |
 |      1       |        0        |  1282 |
 |      1       |        1        |  3918 |
 |      0       |        0        |  9169 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.7708046429274051,
 'log_loss': 0.39189086788834826,
 'precision': 0.7889649617398309,
 'recall': 0.7534615384615385,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+------+-------+
 | threshold | fpr | tpr |  p   |   n   |
 +-----------+-----+-----+------+-------+
 |    0.0    | 1.0 | 1.0 | 5200 | 10217 |
 |   1e-05   | 1.0 | 1.0 | 5200 | 10217 |
 |   2e-05   

In [45]:
best_model.get_current_options#()

<bound method RandomForestClassifier.get_current_options of Class                          : RandomForestClassifier

Schema
------
Number of examples             : 137593
Number of feature columns      : 185
Number of unpacked features    : 185
Number of classes              : 2

Settings
--------
Number of trees                : 150
Max tree depth                 : 12
Training time (sec)            : 126.4014
Training auc                   : 0.9154
Validation auc                 : 0.9032
>

# Boosted Classifier
### HyperParams search

In [87]:
# Boosted Trees HyperParams

# target_col = 'Y'

params = {
    'target': target_col, ## the target column string name 
    'max_iterations': [
                      200,
                       250,
                       90,
                       120
                      ], #The maximum number of iterations for boosting. Each iteration == extra tree.
    'class_weights': [
                      None, 
#                       'auto',
    ], #Weights the examples in the training data according to the given class weights.
#     'class_weights': [None],
    'max_depth': [
        2,
#         3,
        4,
        6,
        9,
    ], #Maximum depth of a tree. Must be at least 1.
    'step_size': [
        1e-1,
        0.5,
#         1e-2,
        1,
    ], # Step size (shrinkage) used in update to prevents overfitting
    'min_loss_reduction': [
        1e-2,
        1,
        10,
    ], #Minimum loss reduction required to make a further partition/split a node during the tree learning
    'min_child_weight': [
#         1e-2,
        2,
        5,
        10,
    ], # Controls the minimum weight of each leaf node . larger values > less overfitting
    'row_subsample': [
        0.5,
        0.75,
    ], #Subsample the ratio of the training set in each iteration of tree construction
    
    'column_subsample': [
#         0.01,
        0.1,
        0.05,
                        ], # Subsample the ratio of the columns in each iteration of tree construction
    #'metric': ['accuracy', 'auc', 'f1_score','recall','precision'], # Performance metric(s) that are tracked during training     
    'metric': ['auc'],
    'random_seed' : int(abs(hash('im not joking...'))%1e6) 
}

model_factory = boosted_trees_classifier.create

In [118]:
# in case we want to use random forest's top features result
n_cols = 30

top_rf_features = True
# top_rf_features = None

if top_rf_features:
    filter_cols = list(best_feat['name'].head(n_cols).to_numpy()) + [target_col]

    X_val_gl = X_val_gl[filter_cols]
    X_gl = X_gl[filter_cols]
    
    kfold = cross_validation.KFold(
    X_gl,num_folds=num_folds)

### Grid Search Boosted 

In [90]:
%%time
start_time = time.time()
model_search = model_parameter_search.create( datasets = kfold, 
                                             #[data, val_set],
                                             model_factory = model_factory,
                                    model_parameters = params, 
                                    perform_trial_run = True  )

search_results = model_search.get_results()

all_time = start_time - time.time()

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-May-25-2017-16-38-0700000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-May-25-2017-16-38-0700000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-May-25-2017-16-38-0700000' already exists. Renaming the job to 'Model-Parameter-Search-May-25-2017-16-38-0700000-8b683'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-May-25-2017-16-38-0700000-8b683' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-May-25-2017-16-38-0700000-8b683' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-May-25-2017-16-38-0700001' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-May-25-2017-16-38

CPU times: user 6.37 s, sys: 2.13 s, total: 8.5 s
Wall time: 3min 24s


In [91]:
res = model_search.get_metrics()
res

task_name,status,start_time,run_time,exception,exception_message,exception_traceback
_train_test_model-0-0,Completed,2017-05-25 16:38:22,4.83546280861,None,None,None
_train_test_model-0-1,Completed,2017-05-25 16:38:27,4.28994393349,None,None,None
_train_test_model-0-2,Completed,2017-05-25 16:38:31,12.6711649895,None,None,None
_train_test_model-0-3,Completed,2017-05-25 16:38:44,6.12934613228,None,None,None
_train_test_model-0-4,Completed,2017-05-25 16:38:50,5.99115419388,None,None,None
_train_test_model-0-5,Completed,2017-05-25 16:38:56,5.96607494354,None,None,None
_train_test_model-0-6,Completed,2017-05-25 16:39:02,6.18768501282,None,None,None
_train_test_model-0-7,Completed,2017-05-25 16:39:08,6.30825185776,None,None,None
_train_test_model-0-8,Completed,2017-05-25 16:39:14,6.13260912895,None,None,None
_train_test_model-0-9,Completed,2017-05-25 16:39:20,6.24866008759,None,None,None


In [93]:
# res['exception_message']

In [94]:
res

task_name,status,start_time,run_time,exception,exception_message,exception_traceback
_train_test_model-0-0,Completed,2017-05-25 16:38:22,4.83546280861,None,None,None
_train_test_model-0-1,Completed,2017-05-25 16:38:27,4.28994393349,None,None,None
_train_test_model-0-2,Completed,2017-05-25 16:38:31,12.6711649895,None,None,None
_train_test_model-0-3,Completed,2017-05-25 16:38:44,6.12934613228,None,None,None
_train_test_model-0-4,Completed,2017-05-25 16:38:50,5.99115419388,None,None,None
_train_test_model-0-5,Completed,2017-05-25 16:38:56,5.96607494354,None,None,None
_train_test_model-0-6,Completed,2017-05-25 16:39:02,6.18768501282,None,None,None
_train_test_model-0-7,Completed,2017-05-25 16:39:08,6.30825185776,None,None,None
_train_test_model-0-8,Completed,2017-05-25 16:39:14,6.13260912895,None,None,None
_train_test_model-0-9,Completed,2017-05-25 16:39:20,6.24866008759,None,None,None


In [110]:
search_results.shape

(10, 17)

In [111]:
search_results.head()

class_weights,column_subsample,early_stopping_rounds,max_depth,max_iterations,metric,min_child_weight
None,0.05,5,2,90,auc,2
None,0.05,5,2,250,auc,2
None,0.1,5,2,120,auc,2
None,0.05,5,6,120,auc,10
None,0.05,5,6,200,auc,2
None,0.1,5,6,200,auc,10
None,0.1,5,2,120,auc,10
None,0.1,5,2,90,auc,10
None,0.1,5,9,90,auc,2
None,0.1,5,9,120,auc,10


In [97]:
for col in [col for col in search_results.column_names() if \
            not('accuracy' in col) and col!= 'model_id' and col!= 'random_seed' and col!= 'fold_id' \
                               and col!= 'num_folds']:
    print(search_results.groupby(col,
                             {'mean':gl.aggregate.MEAN('mean_training_accuracy'),
                                'std':gl.aggregate.STD('mean_training_accuracy')}))

+---------------+-----------------+----------------+
| class_weights |       std       |      mean      |
+---------------+-----------------+----------------+
|      None     | 0.0105991460052 | 0.814007603126 |
+---------------+-----------------+----------------+
[1 rows x 3 columns]

+------------------+------------------+----------------+
| column_subsample |       std        |      mean      |
+------------------+------------------+----------------+
|       0.05       | 0.00541972337919 | 0.80334738611  |
|       0.1        | 0.00643329674533 | 0.821114414471 |
+------------------+------------------+----------------+
[2 rows x 3 columns]

+-----------------------+-----------------+----------------+
| early_stopping_rounds |       std       |      mean      |
+-----------------------+-----------------+----------------+
|           5           | 0.0105991460052 | 0.814007603126 |
+-----------------------+-----------------+----------------+
[1 rows x 3 columns]

+-----------+---------

## hyperparameter results

* class_weights: None, gana por afano
* column_subsample : algo menor a 1 pero no queda claro si cuanto menos mejor
* row_subsample : algo menor a 1 pero no queda claro si cuanto menos mejor, refinar
* max_depth: varia mucho, refinar
* max_iterations: tampoco queda claro si poquito o mucho es mejor, reprobar
* metric: el f1_score es el unico que no parece variar mucho,los demas cambian demasiado
* min_child_weight: varia mucho, reprobar con mas
* min_loss_reduction: varia mucho, reprobar
* step_size: mas grande parece ser mejor> 0.1

### Results

* bootstrap  ???= False es 5% mejor
* min_samples_leaf ????= mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators ???= mas pareceria mejor, pero depende del app y hay que ver 'cuanto' mejora por app
* citerion ???= entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features ???= no afecta al score. con auto esta bien
* max_depth ??=  mas es mejor. intentaria probar con >15

## boosted run with best params

In [112]:
params = model_search.get_best_params()

In [113]:
params

{'class_weights': None,
 'column_subsample': 0.1,
 'early_stopping_rounds': 5,
 'max_depth': 2,
 'max_iterations': 120,
 'metric': 'auc',
 'min_child_weight': 10,
 'min_loss_reduction': 0.01,
 'random_seed': 415168,
 'row_subsample': 0.9,
 'step_size': 1,
 'target': 'target'}

In [107]:
# params = {'class_weights': None,
#  'column_subsample': 0.1,
#  'early_stopping_rounds': 5,
#  'max_depth': 2,
#  'max_iterations': 120,
#  'metric': 'auc',
#  'min_child_weight': 10,
#  'min_loss_reduction': 0.01,
#  'random_seed': 415168,
#  'row_subsample': 0.9,
#  'step_size': 1,
#  'target': 'target'}

In [119]:
%%time
best_model =  boosted_trees_classifier.create(dataset=X_gl,
                                             validation_set = X_val_gl,
                                             **params)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 137593

Number of classes           : 2

Number of feature columns   : 30

...

| 11        | 0.822269     | 0.877986     | 0.872180       |

| 25        | 2.019301     | 0.886901     | 0.880440       |

+-----------+--------------+--------------+----------------+

Early stop triggered. Returning the best model at iteration: 42

CPU times: user 14.3 s, sys: 512 ms, total: 14.8 s
Wall time: 5.12 s


In [124]:
best_feat = best_model.get_feature_importance()
best_feat = list(best_feat['name'].to_numpy())
best_feat[:30]

['TimeWeekDay_IN_VUL_09',
 'CallsWeekDay_IN_09',
 'TimeWeekDay_OUT_VUL_12',
 'MOBILITY_DIAMETER_WEEKNIGHT',
 'COUNT_WEEKNIGHT_0',
 'CallsWeekDay_OUT_09',
 'CallsWeekDay_IN_VUL_08',
 'TimeWeekDay_IN_VUL_08',
 'TimeWeekDay_OUT_08',
 'TimeWeekDay_IN_12',
 'CallsWeekDay_IN_12',
 'CallsWeekDay_OUT_08',
 'TimeWeekDay_IN_10',
 'TimeWeekDay_IN_11',
 'TimeWeekDay_OUT_11',
 'TimeWeekDay_IN_09',
 'TimeWeekDay_OUT_09',
 'COUNT_0',
 'TimeWeekDay_OUT_VUL_08',
 'CallsWeekDay_OUT_VUL_08',
 'COUNT_1',
 'EPIDEMIC',
 'MOBILITY_DIAMETER',
 'TimeWeekDay_OUT_10',
 'TimeWeekDay_OUT_12',
 'CallsWeekDay_IN_08',
 'STATE_Distrito_Federal',
 'STATE_Hidalgo',
 'CallsWeekDay_OUT_12',
 'CallsWeekDay_OUT_10']

In [121]:
test_error = best_model.evaluate(X_val_gl)
test_error

{'accuracy': 0.8327819939028346,
 'auc': 0.8818069525903364,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  957  |
 |      1       |        0        |  1621 |
 |      1       |        1        |  3579 |
 |      0       |        0        |  9260 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.7352095316351686,
 'log_loss': 0.3995800021922013,
 'precision': 0.7890211640211641,
 'recall': 0.6882692307692307,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+------+-------+
 | threshold | fpr | tpr |  p   |   n   |
 +-----------+-----+-----+------+-------+
 |    0.0    | 1.0 | 1.0 | 5200 | 10217 |
 |   1e-05   | 1.0 | 1.0 | 5200 | 10217 |
 |   2e-05   |

In [122]:
best_model.get_current_options#()

<bound method BoostedTreesClassifier.get_current_options of Class                          : BoostedTreesClassifier

Schema
------
Number of examples             : 137593
Number of feature columns      : 30
Number of unpacked features    : 30
Number of classes              : 2

Settings
--------
Number of trees                : 42
Max tree depth                 : 2
Training time (sec)            : 4.033
Training auc                   : 0.889
Validation auc                 : 0.8818
>

# Sklearn 

validation, bias variance, learning curves  and decision tree models

In [36]:
from sklearn.linear_model import *
from sklearn.ensemble import *

from sklearn import model_selection
from sklearn.model_selection import validation_curve

from sklearn.multiclass import OneVsRestClassifier

from sklearn.utils import *

from sklearn.preprocessing import label_binarize, scale, StandardScaler, MinMaxScaler

from sklearn.metrics import *

from sklearn.tree import *

from sklearn.grid_search import *

from sklearn.pipeline import Pipeline

/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## bias variance trade-off in the number of attributes
we first used a random forest to get the best attributes in the model and now will try to overfit the validation set using too many features.

In [37]:
best_columns = best_feat['name'].to_numpy()

X[best_columns].columns

NameError: name 'best_feat' is not defined

## scale numerical features when necessary

In [38]:
# by definition any column which is outside of [-1,1]
scaling_cols = (X.max()) >1 | (X.min()<-1)

In [39]:
scaling_cols = scaling_cols[scaling_cols==True].index

In [40]:
scale = MinMaxScaler(copy=True)

In [41]:
X[scaling_cols] = scale.fit_transform(X[scaling_cols])

/home/juan/mfixman/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [68]:
# define our c range to compare
inv_c_range = np.logspace(-4, 100, 15)
# param_range = 1./inv_c_range
param_range = np.linspace(-3,0.5,30)
# param_range = np.ceil(param_range)
param_range = np.exp((param_range)*np.log(10))
param_range


array([  1.00000000e-03,   1.37382380e-03,   1.88739182e-03,
         2.59294380e-03,   3.56224789e-03,   4.89390092e-03,
         6.72335754e-03,   9.23670857e-03,   1.26896100e-02,
         1.74332882e-02,   2.39502662e-02,   3.29034456e-02,
         4.52035366e-02,   6.21016942e-02,   8.53167852e-02,
         1.17210230e-01,   1.61026203e-01,   2.21221629e-01,
         3.03919538e-01,   4.17531894e-01,   5.73615251e-01,
         7.88046282e-01,   1.08263673e+00,   1.48735211e+00,
         2.04335972e+00,   2.80721620e+00,   3.85662042e+00,
         5.29831691e+00,   7.27895384e+00,   1.00000000e+01])

In [ ]:
%%time
train_scores, test_scores = validation_curve(
    LogisticRegression(solver = 'lbfgs', 
                       max_iter = 100,
                      warm_start=True),
    X, Y, 
    param_name="C", 
    param_range=param_range,
    cv=5, 
    scoring="f1", 
    n_jobs=-1)

Process PoolWorker-84:
Process PoolWorker-92:
Process PoolWorker-93:
Process PoolWorker-85:
Process PoolWorker-87:
Process PoolWorker-95:
Process PoolWorker-86:
Process PoolWorker-82:
Process PoolWorker-96:
Process PoolWorker-97:
Process PoolWorker-91:
Process PoolWorker-83:
Process PoolWorker-88:
Process PoolWorker-90:
Process PoolWorker-94:
Process PoolWorker-89:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/

KeyboardInterrupt


In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

In [ ]:
plt.title("Validation Curve with Logistic Regression Classifier")
plt.xlabel("$Param$")
plt.ylabel(" Score")
# plt.ylim(0.0, 10.1)

lw = 1

plt.semilogx(param_range, train_scores_mean, label="Training score with std-dev band",
             color="navy", lw=lw)

plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="blue", lw=lw)

plt.semilogx(param_range, test_scores_mean, label="Cross-validation score avg",
             color="darkorange", lw=lw)

# plt.fill_between(param_range, test_scores_mean - test_scores_std,
#                  test_scores_mean + test_scores_std, alpha=0.2,
#                  color="red", lw=lw)

plt.legend(loc="best")


In [ ]:
finished = True
finished

## Logistic Regression cross validation

In [54]:

clf =  LogisticRegressionCV(n_jobs =3, class_weight= 'balanced', scoring = 'f1',cv=3)

clf.fit(X_train,y_train)


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.67      0.85      0.75     43545
          1       0.40      0.19      0.25     22464

avg / total       0.58      0.63      0.58     66009

             precision    recall  f1-score   support

          0       0.64      0.75      0.69     83353
          1       0.40      0.28      0.33     48666

avg / total       0.55      0.58      0.56    132019

             precision    recall  f1-score   support

          0       0.63      0.67      0.65    124185
          1       0.39      0.35      0.37     73843

avg / total       0.54      0.55      0.55    198028

             precision    recall  f1-score   support

          0       0.62      0.56      0.59    164393
          1       0.38      0.44      0.41     99645

avg / total       0.53      0.52      0.52    264038

             precision    recall  f1-score   support

          0       0.64      0.66      0.65    415477
          1       0.39      0.36 

AttributeError: 'LogisticRegressionCV' object has no attribute 'grid_scores_'

# REST of shit

### Count Featurizer on Antennas
nos quedamos con las columnas de antennas y en graphlab aplicamos el algo de CountFeaturizer para cada categoria de
Antenna_ID_N


In [ ]:
%%time
import graphlab as gl
from graphlab.toolkits.feature_engineering import *
ant_cols = [col for col in X_train.columns if "ANTENNA_ID" in col]
ant_sframe_fit = gl.SFrame(X_fit[ant_cols + ['y']])
ant_sframe_train = gl.SFrame(X_train[ant_cols + ['y']])
ant_sframe_val = gl.SFrame(X_val[ant_cols + ['y']])

In [ ]:
%%time
countfeat = gl.feature_engineering.create(ant_sframe_fit, 
               CountFeaturizer(target='y'))

# Transform the train set. This is the dataset I will train my classifier on
transformed_ant_train = countfeat.transform(ant_sframe_train)
transformed_ant_val = countfeat.transform(ant_sframe_val)

del ant_sframe_fit,ant_sframe_train,ant_sframe_val

#por alguna razon guarda los valores de probabilidad como una lista de un unico valor
for col in [col for col in transformed_ant_train.column_names() if "prob_" in col]:
    transformed_ant_train[col] = transformed_ant_train[col].apply(lambda x: x[0]) 
    transformed_ant_val[col] = transformed_ant_val[col].apply(lambda x: x[0]) 

#me quedo solo con los valores de probabilidad.
transformed_ant_train = transformed_ant_train[[col for col in transformed_ant_train.column_names() if "prob_" in col]]
transformed_ant_val = transformed_ant_val[[col for col in transformed_ant_train.column_names() if "prob_" in col]]

transformed_ant_val = transformed_ant_val.to_dataframe()
transformed_ant_train = transformed_ant_train.to_dataframe()

In [ ]:
X_val[ant_cols] = transformed_ant_val.values
X_train[ant_cols] = transformed_ant_train.values
del transformed_ant_train, transformed_ant_val

### Cross Validate SGD

In [ ]:
param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]

In [37]:
start_time = time.time()

param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]
sgd = SGDClassifier(loss='modified_huber', penalty='elasticnet', 
             fit_intercept=True,  shuffle=True, 
                    n_jobs=3,learning_rate='optimal', power_t =2, eta0 =5,
                    class_weight='balanced', average=40)

clf =GridSearchCV(sgd, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)



predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


...

[CV]  n_iter=2, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.122741 -   3.5s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.132433 -   3.7s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135111 -   3.4s
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135908 -   3.1s


[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:  1.7min finished


             precision    recall  f1-score   support

          0       1.00      0.31      0.47    355992
          1       0.06      0.99      0.11     15152

avg / total       0.96      0.33      0.45    371144

This cell took 108.56432461738586 seconds to run


In [ ]:
threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))

In [38]:
clf.best_estimator_

SGDClassifier(alpha=0.005, average=40, class_weight='balanced', epsilon=0.1,
       eta0=5, fit_intercept=True, l1_ratio=0.0006000000000000001,
       learning_rate='optimal', loss='modified_huber', n_iter=3, n_jobs=3,
       penalty='elasticnet', power_t=2, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [39]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_iter
2.0    0.125741
3.0    0.132943
Name: mean_score, dtype: float64
n_iter
2.0    0.002609
3.0    0.002703
Name: mean_score, dtype: float64


### Results
* l1_ratio = cuanto mas chico mejor con lo cual la perdida l2 parece ser mejor
* alpha = 1e-3 es suficiente pues casi no afecta el score
* power_t = muy variado, no parece haber correlacion entre el tamanyo y el avg, mean_score
* eta0 = no afecta mucho pero parece ser que con ser >1 ya esta
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

In [51]:
len(predicted_proba)

371161

## Cross Validate NN

In [35]:
start_time = time.time()

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

param_grid = { 'alpha': [1e-1,5*1e-2,1e-2],
              'hidden_layer_sizes':[(50,),(15,5)],
              'learning_rate': ['adaptive',"invscaling"],
              "algorithm": ['adam'],'momentum': [1e-2, 1e-1, 0.5],
  'power_t': [1e-3, 5*1e-4, 1e-5], 'activation':['logistic','relu']
 }

mlp = MLPClassifier(shuffle=True, 
                 verbose=True)

clf =GridSearchCV(mlp, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 
Iteration 1, loss = 0.20405210
Iteration 1, loss = 0.20380843
Iteration 1, loss = 0.20378706
Iteration 1, loss = 0.20405210


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.5s
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.7s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   2.1s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.9s


KeyboardInterrupt: 

In [1]:
clf.best_estimator_

NameError: name 'clf' is not defined

In [52]:
# compare best parameters to tune
coln=1
cv_result.groupby(cv_result.columns[coln])['mean_score'].mean()
cv_result.groupby(cv_result.columns[coln])['mean_score'].std()

alpha
0.00     0.000007
0.01     0.000007
0.10     0.000007
0.50     0.000007
1.00     0.000007
10.00    0.000007
Name: mean_score, dtype: float64

### Results
* activation = logistic es 10% mejor
* alpha = 1e-2 el mejor 
* power_t = cuanto mas chico mejor, 1e-3 por lo menos
* hidden_layer_size = menos layers es mejor..?
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

## Bernoulli RBM features selection & Logit crossval

In [127]:
start_time = time.time()

## obs. este metodo es especial y asume que todos los valores son True/False o que \in [0,1]
# luego tengo que pensar en como tomar los features nuevamente.
# tampoco 'fittea' en la forma tradicional. Sino que se le puede tomar al y como un feature mas y esta red
# va 'modificando' todos los valores del X (minimizando la entropia) para dar un output. Luego corriendo 
# clf.gibbs(X_test) con el y_test como feature tmb, nos transforma la data para ver el output como la 'prediccion'
# obviamente no tiene probabilidades


#X = X_train[X_cols].values
#y = X_train['ground_truth'].values

df = X_train.drop(X_train[X_train[X_train.columns[0:3]].\
                                   sum(axis=1)==0].index)
df = df[X_cols + ['ground_truth']]

for col in X_train.columns[0:3]:
    df[col] = df[col]*1.0/df[df.columns[0:3]].sum(axis=1)

df[df.columns[3]] =  df[df.columns[3]]/df[df.columns[3]].max()
df[df.columns[4]] =  df[df.columns[4]]/df[df.columns[4]].max()

X = df[df.columns[:-1]].values
y = df['ground_truth'].values


param_grid = {'rbm__n_components': [256, 128,46,10],
   'rbm__n_iter':[15,10,5], 'rbm__learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1],
  'rbm__batch_size': [10e4,3*10e3, 1e3, 300],"logistic__C": [1.0, 10.0, 100.0] 
 }

rbm = BernoulliRBM(verbose=True)
logistic = LogisticRegression()
classifier = Pipeline([("rbm", rbm), ("logistic", logistic)])

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

clf =GridSearchCV(classifier, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

#este metodo NO tiene predicted proba, lo que hacemos es recorrer 
#predicted_labels = rbm.gibbs(X_test)[:,-1]
#real_labels = X_test[:,-1]
#print(classification_report(real_labels,predicted_labels ))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

finished = True

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
    
print('This cell took %s seconds to run' % all_time)


KeyboardInterrupt: 

In [ ]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

In [23]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


criterion
gini    0.802919
Name: mean_score, dtype: float64
criterion
gini    0.000598
Name: mean_score, dtype: float64


## CrossV SVC

In [ ]:
start_time = time.time()

param_grid = [{'kernel': ['rbf'], 'gamma': list(10.0 ** np.arange(-2, 3)),
                     'C': list(10.0 ** np.arange(0, 4))},
                    {'kernel': ['poly'], 'C': list(10.0 ** np.arange(0, 4)), 'degree'[2,3,4]}]

svc = SVC(shuffle=True, probability=True,decision_function_shape = 'ovr',
           verbose=True, class_weight='balanced'
          )

clf =GridSearchCV(svc, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

In [ ]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


In [214]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

#model = model.fit(X,y,sample_weight=W)

W = np.array([10 if i == 1 or i ==2  else 1 for i in y_mini])
gradboost.fit(X_mini,y_mini, sample_weight=W)

elapsed_time =   time.time() - start_time
print('GradientBoosting took %s seconds to run' % elapsed_time)

#validated =  cross_val_score(gradboost,X,y,cv=5, scoring = "f1_weighted")


all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

GradientBoosting took 117.26272392272949 seconds to run
This cell took 117.26284193992615 seconds to run


## Timings
* 5s con  10 n_estimadores, 15 max_depth y X.sample(0.2)
* 15s con 10 n_estimadores, 15 max_depth y X.sample(0.2)
* 47s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 35s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 117 con 20 n_estimadores, 20 max_depth y X.sample(0.4)

In [31]:
X_train['ground_truth'].value_counts()

0    128153
1      7064
Name: ground_truth, dtype: int64

### Results
* NO escalar (normalizar, restar la media  dividir por la std, etc) los resultados pues REVIENTAN los scores.
* bootstrap  = False es 5% mejor
* min_samples_leaf = mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators = aumentar mas de 30 no tendria mucho sentido
* citerion = entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features = no afecta al score. con auto esta bien
* max_depth =  mas es mejor. intentaria probar con >15

In [ ]:
%%time
#con decomposicion qr tratamos de ver si existe alguna columna que sea comb linear de las demas
linear_test = np.linalg.qr(X_train.sample(1e6))[1]
#notar que devuelve la tabla de tamanyo N`columnas x Ncolumnas

#sumo a traves de las columnas para que me de el valor absoluto sumado de c/fila
linear_test = abs(linear_test.sum(axis=1))<1e-2
#si hubiese alguan que sea linearcomb entonces tendria que aparecer que toda la fila es de ceros
for i, col in enumerate(X_train.columns):
    if linear_test[i] == True:
        print(col)

# Todo
* evaluate hit_rate and 
* tune adaboost, bernoulliRBM
* xgboost
* libffm
* SVC muy lento.. speed up in AWS
